In [1]:
import torch
import yfinance as yf
import pandas as pd
import os
import requests
import datetime
import ray
from ray import tune, air

import sys
sys.path.append('./data')
sys.path.append('./training')
import data, training


torch.cuda.is_available()

True

In [3]:
BATCH_SIZE=96
WINDOW_SIZE=32
NUM_WORKERS=6
N_FEATURES=1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
N_TRIES=8 # Increase to try a larger search space

'cuda'

In [4]:
ray.init(num_cpus=int(os.cpu_count() * .75), num_gpus=1)

2023-04-12 18:24:58,836	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


In [5]:
# Need to go requests route to avoid forbidden error
url = 'https://stockanalysis.com/list/biggest-companies/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)

symbols = list(dfs[0]['Symbol'].iloc[:25].values)

In [6]:
# Maximum number of days for YFinance 5m data is 60, but it only takes 59 day diff
end_date = datetime.date.today() 
start_date = end_date - datetime.timedelta(days=59)

dataset = yf.download(
    symbols,
    period='max', 
    interval='5m',
    start=start_date, 
    end=end_date
)['Adj Close']

train, val = data.get_train_val_dicts(dataset)

[*********************100%***********************]  25 of 25 completed

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted


In [7]:
config = {
    'device': DEVICE,
    'D_MODEL': tune.choice([32, 64]),
    'N_HEADS': tune.choice([2, 8]),
    'D_FF': tune.choice([128, 256]),
    'DROPOUT': tune.choice([0., .05, .1, .15]),
    'ACTIVATION': tune.choice(['relu', 'gelu']),
    'NUM_LAYERS': tune.choice([3, 5]),
    'LR': tune.choice([1e-3, 1e-4, 1e-5]),
    'WINDOW_SIZE': tune.choice([32, 64])
}
param_space = {'train_loop_config': config}

In [8]:
trainer, tune_config = training.setup_ray(train, val, BATCH_SIZE, param_space, n_tries=N_TRIES)

tuner = tune.Tuner(
    trainer,
    run_config=air.RunConfig(
        local_dir="./tfmr_logger/",
    ),
    tune_config=tune_config,
)

result = tuner.fit()

2023-04-12 18:25:02,527	WARNING bohb_search.py:152 -- You passed a `space` parameter to <class 'ray.tune.search.bohb.bohb_search.TuneBOHB'> that contained unresolved search space definitions. <class 'ray.tune.search.bohb.bohb_search.TuneBOHB'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.


Trial name,status,loc,train_loop_config...,train_loop_config...,train_loop_config...,train_loop_config...,train_loop_config/LR,train_loop_config...,train_loop_config...,train_loop_config...,iter,total time (s),train_loss,val_loss,_timestamp
TorchTrainer_1e8c9764,RUNNING,10.0.0.153:6288,gelu,0,256,64,0.0001,3,8,32,,,,,
TorchTrainer_de38ff04,PAUSED,10.0.0.153:4922,gelu,0.05,128,64,0.001,3,2,64,3,17.147,137.254,133.518,1681338323
TorchTrainer_e08cee14,PAUSED,10.0.0.153:5085,relu,0,256,64,1e-05,3,2,64,3,15.6769,136.741,133.34,1681338343
TorchTrainer_eb48e66e,PAUSED,10.0.0.153:5258,gelu,0,256,64,0.0001,5,8,32,3,17.972,55.4542,53.7218,1681338365
TorchTrainer_f735046c,PAUSED,10.0.0.153:5709,relu,0.1,256,32,0.0001,5,8,32,3,18.1625,56.1481,54.1533,1681338388
TorchTrainer_0451cab8,PAUSED,10.0.0.153:5900,gelu,0.15,128,32,1e-05,5,2,32,3,17.6434,56.2803,54.3783,1681338409
TorchTrainer_116ed93e,PAUSED,10.0.0.153:6091,gelu,0.05,256,32,0.0001,5,2,32,3,17.9782,56.2731,54.3535,1681338431
TorchTrainer_2baa1e58,PENDING,,relu,0.05,256,64,0.001,5,8,32,,,,,


(RayTrainWorker pid=4961) 2023-04-12 18:25:09,151	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=4961) wandb: wandb version 0.14.2 is available!  To upgrade, please run:
(RayTrainWorker pid=4961) wandb:  $ pip install wandb --upgrade
(RayTrainWorker pid=4961) wandb: Tracking run with wandb version 0.13.4
(RayTrainWorker pid=4961) wandb: Run data is saved locally in /media/alexander/38F66E50F66E0E86/Users/Alexander/Downloads/Python Projects/medium_articles/bayesian_opt_hyperband/tfmr_logger/TorchTrainer_2023-04-12_18-25-02/TorchTrainer_de38ff04_1_ACTIVATION=gelu,DROPOUT=0.0500,D_FF=128,D_MODEL=64,LR=0.0010,NUM_LAYERS=3,N_HEADS=2,WINDOW_SIZE=64_2023-04-12_18-25-02/rank_0/wandb/run-20230412_182511-1le3nxpz
(RayTrainWorker pid=4961) wandb: Run `wandb offline` to turn off syncing.
(RayTrainWorker pid=4961) wandb: Syncing run hopeful-thunder-71
(RayTrainWorker pid=4961) wandb: ⭐️ View project at https://wandb.ai/lavellea/torch_stock_trans

Result for TorchTrainer_de38ff04:
  _time_this_iter_s: 11.315113306045532
  _timestamp: 1681338321
  _training_iteration: 1
  date: 2023-04-12_18-25-21
  done: false
  experiment_id: da2d6e69274644179338662861911a63
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 4922
  should_checkpoint: true
  time_since_restore: 15.069329738616943
  time_this_iter_s: 15.069329738616943
  time_total_s: 15.069329738616943
  timestamp: 1681338321
  timesteps_since_restore: 0
  train_loss: 139.45068081942472
  training_iteration: 1
  trial_id: de38ff04
  val_loss: 134.7938232421875
  warmup_time: 0.0027675628662109375
  


100%|██████████| 11/11 [00:00<00:00, 33.74it/s]
(RayTrainWorker pid=4961) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=4961)   return torch._transformer_encoder_layer_fwd(
100%|██████████| 11/11 [00:00<00:00, 34.43it/s]
(RayTrainWorker pid=4961) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=4961)   return torch._transformer_encoder_layer_fwd(
(RayTrainWorker pid=4961) Exception in t

...

100%|██████████| 21/21 [00:00<00:00, 33.87it/s]
(RayTrainWorker pid=5753) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=5753)   return torch._transformer_encoder_layer_fwd(
100%|██████████| 21/21 [00:00<00:00, 33.01it/s]
(RayTrainWorker pid=5753) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=5753)   return torch._transformer_encoder_layer_fwd(
(RayTrainWorker pid=5753) Exception in t

Result for TorchTrainer_0451cab8:
  _time_this_iter_s: 10.52669382095337
  _timestamp: 1681338405
  _training_iteration: 1
  date: 2023-04-12_18-26-46
  done: false
  experiment_id: 9ecbf16fee1d4f1d9a3cbb2254514a0b
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 5900
  should_checkpoint: true
  time_since_restore: 14.16873550415039
  time_this_iter_s: 14.16873550415039
  time_total_s: 14.16873550415039
  timestamp: 1681338406
  timesteps_since_restore: 0
  train_loss: 57.40310886928013
  training_iteration: 1
  trial_id: 0451cab8
  val_loss: 54.84373528616769
  warmup_time: 0.0027167797088623047
  


100%|██████████| 21/21 [00:00<00:00, 38.44it/s]
(RayTrainWorker pid=5949) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=5949)   return torch._transformer_encoder_layer_fwd(
100%|██████████| 21/21 [00:00<00:00, 36.85it/s]
(RayTrainWorker pid=5949) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=5949)   return torch._transformer_encoder_layer_fwd(
(RayTrainWorker pid=5949) Exception in t

Result for TorchTrainer_116ed93e:
  _time_this_iter_s: 10.730890989303589
  _timestamp: 1681338428
  _training_iteration: 1
  date: 2023-04-12_18-27-08
  done: false
  experiment_id: 34d96ac2f649468482f48f8d0a42f5fc
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 6091
  should_checkpoint: true
  time_since_restore: 14.395349264144897
  time_this_iter_s: 14.395349264144897
  time_total_s: 14.395349264144897
  timestamp: 1681338428
  timesteps_since_restore: 0
  train_loss: 57.08022617158436
  training_iteration: 1
  trial_id: 116ed93e
  val_loss: 54.69199752807617
  warmup_time: 0.0027391910552978516
  


100%|██████████| 21/21 [00:00<00:00, 36.80it/s]
(RayTrainWorker pid=6138) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=6138)   return torch._transformer_encoder_layer_fwd(
100%|██████████| 21/21 [00:00<00:00, 36.54it/s]
(RayTrainWorker pid=6138) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=6138)   return torch._transformer_encoder_layer_fwd(
(RayTrainWorker pid=6138) Exception in t